# Сбор, разметка и предобработка данных

## Исправление путей

In [1]:
import sys
import os
import warnings
from importlib import import_module

# Получаем текущую рабочую директорию ноутбука
notebook_dir = os.getcwd()
print(f"Текущая директория ноутбука: {notebook_dir}")

# Переходим на один уровень вверх, чтобы получить корневую директорию проекта
project_root = os.path.abspath(os.path.join(notebook_dir, os.pardir))
print(f"Корневая директория проекта: {project_root}")

# Добавляем корневую директорию проекта в sys.path, если ее там еще нет
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Добавлено в sys.path: {project_root}")
else:
    print(f"{project_root} уже в sys.path.")

Текущая директория ноутбука: /Users/maksimlyara/Documents/GitHub/DL_NLP_task/notebooks
Корневая директория проекта: /Users/maksimlyara/Documents/GitHub/DL_NLP_task
Добавлено в sys.path: /Users/maksimlyara/Documents/GitHub/DL_NLP_task


## Импорты

In [2]:
## Импорты
import pandas as pd
import numpy as np
import multiprocessing
import math
import time
import logging
import logging.handlers
from IPython.display import display
from tqdm.auto import tqdm
import traceback
from collections import Counter
import ast

import google.generativeai as genai
from src.workers.gemini_workers import (
    _process_chunk_with_token_batches_for_worker,
    PROMPT_INSTRUCTION_TEMPLATE,
)
from src.utils.mp_helpers import worker_initializer
import json
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

## Глобальные переменные

In [3]:
DATA_FOR_LLM_PATH = os.path.join(project_root, 'data', 'external', 'data_for_llm', 'RIA_before_2025.csv')
DATA_RAW_PATH = os.path.join(project_root, 'data', 'raw', 'filtered_news.csv')
DATA_PREPROCESSED_PATH = os.path.join(project_root, 'data', 'preprocessed', 'preprocessed_news.csv')

API_KEY_FILE = os.path.join(project_root, "src", "config.py") # файл с API ключами

LIST_OF_YEARS = [2023, 2024] # года для фильтрации (доступные данные с 2020 года по 2024 включительно, но рекомендуется выбирать не более 2-3 лет т.к. можно выйти за рамки бесплатного рейт лимита gemini)
DEFAULT_MODEL_NAME = "gemini-2.5-flash-preview-05-20" # модель для генерации меток
TARGET_TOKENS_PER_BATCH = 10000 # количество токенов в батче
DAILY_REQUEST_LIMIT_PER_KEY = 450 # количество запросов в день для каждого ключа
REQUEST_DELAY_SECONDS = 2.23 # задержка между запросами
ROWS_PER_CHUNK = 2000 # количество строк в чанке
PARTIAL_RESULTS_FILE_PATH = os.path.join(project_root, "data", "raw", "processed_news_partial.pkl") # файл для сохранения прогресса

MIN_HIER_LABEL_COUNT = 50 # если меньше N элементов в иерархическом классе отнесётся к "другому"

MODEL_NAME_EMBEDDINGS = "sergeyzh/BERTA" # Модель для получения эмбеддингов
SIMILARITY_THRESHOLD = 0.8 # Порог косинусного сходства для объединения классов
MIN_SUBTHEME_COUNT_FOR_MERGE_TARGET = 10 # Не объединять В подтему класс, если он сам по себе очень редкий (меньше N элементов внутри него)

## Функция для логгирования

In [4]:
MAIN_LOG_FORMAT = '%(asctime)s - %(levelname)s - %(name)s - %(message)s'
MAIN_LOG_DATEFMT = '%Y-%m-%d %H:%M:%S'

In [5]:
def setup_main_logging(log_queue):
    warnings.filterwarnings("ignore", category=FutureWarning, module="numpy._core.fromnumeric", message=".*swapaxes.*")
    warnings.filterwarnings("ignore", category=UserWarning, module="google.protobuf.symbol_database")
    root = logging.getLogger()
    if root.handlers:
        for handler in root.handlers[:]:
            root.removeHandler(handler)
            handler.close()
    root.setLevel(logging.INFO)
    console_handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter(MAIN_LOG_FORMAT, datefmt=MAIN_LOG_DATEFMT)
    console_handler.setFormatter(formatter)
    root.addHandler(console_handler)
    listener = logging.handlers.QueueListener(log_queue, console_handler)
    listener.start()
    return root, listener

In [6]:
# Загрузка API ключей
API_KEYS = [] # Инициализируем на случай ошибки
try:
    relative_path_to_module_file = os.path.relpath(API_KEY_FILE, project_root)
    module_path_without_extension = os.path.splitext(relative_path_to_module_file)[0]
    python_module_name_for_config = module_path_without_extension.replace(os.sep, '.')

    config_module = import_module(python_module_name_for_config)
    API_KEYS = config_module.API_KEYS

    if not API_KEYS or not isinstance(API_KEYS, list) or not all(isinstance(key, str) for key in API_KEYS):
        raise ValueError("API_KEYS должен быть непустым списком строк в файле конфигурации.")
    print(f"Загружено {len(API_KEYS)} API ключей из {API_KEY_FILE}.")

except ModuleNotFoundError as e:
    error_msg = (
        f"Критическая ошибка: Не удалось загрузить конфигурацию API ключей (модуль не найден).\n"
        f"  Файл: '{API_KEY_FILE}'.\n"
        f"  Попытка импорта модуля: '{python_module_name_for_config}'.\n"
        f"  Убедитесь, что файл существует и все директории в пути (например, 'src') содержат файл __init__.py.\n"
        f"  Исходная ошибка: {e}"
    )
    print(error_msg)
    raise RuntimeError(error_msg) from e
except AttributeError as e:
    error_msg = (
        f"Критическая ошибка: Переменная 'API_KEYS' не найдена в модуле '{python_module_name_for_config}' (файл: {API_KEY_FILE}).\n"
        f"  Убедитесь, что в файле {API_KEY_FILE} определен список: API_KEYS = ['key1', 'key2', ...].\n"
        f"  Исходная ошибка: {e}"
    )
    print(error_msg)
    raise RuntimeError(error_msg) from e
except ValueError as e: # Перехватываем ValueError, который мы можем сгенерировать сами
    error_msg = f"Критическая ошибка: Неверный формат API ключей в {API_KEY_FILE}. {e}"
    print(error_msg)
    raise RuntimeError(error_msg) from e
except Exception as e: # Общий обработчик для других неожиданных ошибок
    error_msg = (
        f"Критическая ошибка: Непредвиденная ошибка при загрузке API_KEYS из {API_KEY_FILE}.\n"
        f"  Попытка импорта модуля: '{getattr(sys.modules[__name__], 'python_module_name_for_config', 'не определено')}'\n"
        f"  Исходная ошибка: {e}"
    )
    print(error_msg)
    raise RuntimeError(error_msg) from e

Загружено 5 API ключей из /Users/maksimlyara/Documents/GitHub/DL_NLP_task/src/config.py.


# Gemini

In [7]:
try:
    # Используем первый ключ для этой одноразовой операции
    genai.configure(api_key=API_KEYS[0])
    temp_model_for_counting = genai.GenerativeModel(model_name=DEFAULT_MODEL_NAME)
    base_prompt_for_counting_main = PROMPT_INSTRUCTION_TEMPLATE.format(news_json_payload="[]")
    TOKENS_FOR_BASE_PROMPT_MAIN = temp_model_for_counting.count_tokens(base_prompt_for_counting_main).total_tokens
    print(f"Токены, занимаемые базовой инструкцией промпта (посчитано в основном потоке): {TOKENS_FOR_BASE_PROMPT_MAIN}")
    del temp_model_for_counting # Удаляем временную модель
except Exception as e:
    print(f"Не удалось посчитать токены для базового промпта в основном потоке: {e}. Используем примерное значение 350.")
    TOKENS_FOR_BASE_PROMPT_MAIN = 350

Токены, занимаемые базовой инструкцией промпта (посчитано в основном потоке): 498


In [ ]:
if __name__ == '__main__':
    multiprocessing.freeze_support() 

    try:
        multiprocessing.set_start_method('spawn', force=True) 
    except RuntimeError:
        pass 

    ctx = multiprocessing.get_context('spawn')
    log_queue = ctx.Queue(-1)
    main_logger, log_listener = setup_main_logging(log_queue)

    if not API_KEYS:
        main_logger.critical("API_KEYS не загружены. Обработка невозможна.")
    else:
        # --- Этап 1: Загрузка и фильтрация данных ---
        main_logger.info("--- Этап 1: Загрузка и фильтрация данных ---")
        df_temp_for_filtering = pd.read_csv(DATA_FOR_LLM_PATH, usecols=['date', 'text'])
        main_logger.info(f"  Загружен исходный DataFrame, строк: {len(df_temp_for_filtering):,}")
        df_temp_for_filtering['date'] = pd.to_datetime(df_temp_for_filtering['date'], errors='coerce')
        initial_rows_before_dropna = len(df_temp_for_filtering)
        df_temp_for_filtering.dropna(subset=['date', 'text'], inplace=True)
        main_logger.info(f"  Строк после удаления NaT/пустых текстов: {len(df_temp_for_filtering):,} (удалено: {initial_rows_before_dropna - len(df_temp_for_filtering):,})")
        df_filtered_by_year = df_temp_for_filtering[df_temp_for_filtering['date'].dt.year.isin(LIST_OF_YEARS)].copy()
        main_logger.info(f"  Строк после фильтрации по годам ({LIST_OF_YEARS}): {len(df_filtered_by_year):,}")
        if not df_filtered_by_year.empty:
            df_main_full = df_filtered_by_year[['text']].copy()
        else:
            df_main_full = pd.DataFrame(columns=['text'])
        df_main_full = df_main_full.reset_index(drop=True)
        df_main_full.index.name = 'id'
        main_logger.info(f"  Подготовлен DataFrame для обработки, строк: {len(df_main_full):,}")
        main_logger.info("--- Завершена загрузка и фильтрация данных ---\n")

        # --- Этап 2: Загрузка ранее обработанных результатов ---
        main_logger.info("--- Этап 2: Загрузка ранее обработанных результатов ---")
        # ... (ваш код загрузки df_overall_results и df_to_process_now) ...
        # Упрощенный вариант для примера
        if os.path.exists(PARTIAL_RESULTS_FILE_PATH):
            main_logger.info(f"  Найден файл: {PARTIAL_RESULTS_FILE_PATH}.")
            df_overall_results = pd.read_pickle(PARTIAL_RESULTS_FILE_PATH)
            # Проверка и создание колонок, если их нет
            for col in ["multi_labels", "hier_label"]:
                if col not in df_overall_results.columns:
                    df_overall_results[col] = pd.Series([[] for _ in range(len(df_overall_results))], index=df_overall_results.index, dtype=object)
                else: # Гарантируем, что существующие колонки - списки
                    df_overall_results[col] = df_overall_results[col].apply(lambda x: x if isinstance(x, list) else [])
            
            if 'multi_labels' in df_overall_results.columns and not df_overall_results.empty:
                 processed_ids = set(df_overall_results.index[df_overall_results['multi_labels'].apply(lambda x: isinstance(x, list) and len(x) > 0)])
                 main_logger.info(f"  Загружено {len(processed_ids):,} ранее обработанных ID.")
                 df_to_process_now = df_main_full[~df_main_full.index.isin(processed_ids)].copy()
            else:
                 main_logger.warning("  Колонка 'multi_labels' отсутствует или DataFrame пуст. Обработка всех данных.")
                 df_to_process_now = df_main_full.copy()
                 df_overall_results = df_main_full.copy() # Пересоздаем
                 df_overall_results["multi_labels"] = pd.Series([[] for _ in range(len(df_overall_results))], index=df_overall_results.index, dtype=object)
                 df_overall_results["hier_label"] = pd.Series([[] for _ in range(len(df_overall_results))], index=df_overall_results.index, dtype=object)
        else:
            main_logger.info(f"  Файл {PARTIAL_RESULTS_FILE_PATH} не найден. Начинаем с нуля.")
            df_to_process_now = df_main_full.copy()
            df_overall_results = df_main_full.copy() 
            df_overall_results["multi_labels"] = pd.Series([[] for _ in range(len(df_overall_results))], index=df_overall_results.index, dtype=object)
            df_overall_results["hier_label"] = pd.Series([[] for _ in range(len(df_overall_results))], index=df_overall_results.index, dtype=object)
        main_logger.info("--- Завершена загрузка ранее обработанных результатов ---\n")


        if df_to_process_now.empty and not df_main_full.empty:
            main_logger.info(">>> Все новости уже обработаны. <<<")
        elif df_main_full.empty:
            main_logger.info(">>> Нет данных для обработки (df_main_full пуст). <<<")
        else:
            main_logger.info(f"--- Этап 3: Подготовка к пакетной обработке ---")
            main_logger.info(f"  Всего новостей для обработки: {len(df_to_process_now):,}")
            num_chunks = 0
            if ROWS_PER_CHUNK > 0 and len(df_to_process_now) > 0:
                num_chunks = math.ceil(len(df_to_process_now) / ROWS_PER_CHUNK)
            elif len(df_to_process_now) > 0:
                 num_chunks = 1
            chunks_dfs = []
            if num_chunks > 0:
                if num_chunks == 1: chunks_dfs = [df_to_process_now.copy()]
                else: chunks_dfs = [df_chunk.copy() for df_chunk in np.array_split(df_to_process_now, num_chunks)]
                main_logger.info(f"  Данные разделены на {len(chunks_dfs)} чанков.")
            else:
                main_logger.info("  Нет данных для разделения на чанки (df_to_process_now пуст).")
            main_logger.info("--- Завершена подготовка к пакетной обработке ---\n")

            if chunks_dfs: 
                worker_args = []
                for i, chunk_df_arg in enumerate(chunks_dfs):
                    api_key_for_chunk = API_KEYS[i % len(API_KEYS)]
                    worker_args.append((
                        chunk_df_arg, api_key_for_chunk, i + 1, DEFAULT_MODEL_NAME,
                        TARGET_TOKENS_PER_BATCH, DAILY_REQUEST_LIMIT_PER_KEY, REQUEST_DELAY_SECONDS
                    ))

                pool_size = min(len(API_KEYS), len(chunks_dfs), multiprocessing.cpu_count())
                pool_size = max(1, pool_size)
                main_logger.info(f"--- Этап 4: Запуск многопроцессорной обработки (Pool) ---")
                main_logger.info(f"  Конфигурация: {pool_size} воркеров, {len(chunks_dfs)} чанков.")
                
                start_total_processing_time = time.time()
                processed_chunks_count = 0 # Счетчик для сохранения прогресса
                try:
                    with ctx.Pool(processes=pool_size, initializer=worker_initializer, initargs=(log_queue,)) as pool:
                        # Используем imap_unordered для получения результатов по мере их готовности
                        # и оборачиваем в tqdm для прогресс-бара
                        results_iterator = pool.imap_unordered(_process_chunk_with_token_batches_for_worker, worker_args)
                        
                        for processed_chunk_df in tqdm(results_iterator, total=len(worker_args), desc="Обработка чанков", unit="чанк", file=sys.stderr):
                            processed_chunks_count += 1
                            if processed_chunk_df is not None and not processed_chunk_df.empty:
                                common_indices = df_overall_results.index.intersection(processed_chunk_df.index)
                                if not common_indices.empty:
                                    for col_to_update in ["multi_labels", "hier_label"]:
                                        if col_to_update in processed_chunk_df.columns:
                                            df_overall_results.loc[common_indices, col_to_update] = processed_chunk_df.loc[common_indices, col_to_update]
                            elif processed_chunk_df is None:
                                 main_logger.warning(f"  Воркер для одного из чанков вернул None (чанк #{processed_chunks_count}).")
                            
                            # Сохранение прогресса после обработки каждого чанка (или группы чанков)
                            # Например, каждые 'save_every_n_chunks' или после каждого
                            save_every_n_chunks = max(1, pool_size) # Сохраняем примерно после каждого "супер-батча"
                            if processed_chunks_count % save_every_n_chunks == 0 or processed_chunks_count == len(worker_args):
                                try:
                                    df_overall_results.to_pickle(PARTIAL_RESULTS_FILE_PATH)
                                    main_logger.info(f"  Прогресс сохранен (обработано {processed_chunks_count}/{len(worker_args)} чанков)")
                                except Exception as e_save_interim:
                                    main_logger.error(f"  Ошибка промежуточного сохранения: {e_save_interim}")
                    
                    # Финальное сохранение, если что-то обрабатывалось
                    if processed_chunks_count > 0 :
                        try:
                            df_overall_results.to_pickle(PARTIAL_RESULTS_FILE_PATH)
                            main_logger.info(f"  Итоговый прогресс сохранен в {PARTIAL_RESULTS_FILE_PATH}.")
                        except Exception as e_save_final:
                            main_logger.error(f"  Ошибка финального сохранения: {e_save_final}", exc_info=True)

                except Exception as e_pool:
                    main_logger.error(f"  ОШИБКА ПУЛА ВОРКЕРОВ: {e_pool}", exc_info=True)
                
                end_total_processing_time = time.time()
                total_time_seconds = end_total_processing_time - start_total_processing_time
                total_time_formatted = time.strftime("%H:%M:%S", time.gmtime(total_time_seconds))
                main_logger.info(f"\n--- Завершена многопроцессорная обработка ---")
                main_logger.info(f"  Общее время обработки: {total_time_formatted} ({total_time_seconds:.2f} секунд)")
            else:
                main_logger.info("--- Нет чанков для обработки (Этап 4 пропущен). ---")

        # --- Этап 5: Итоги ---
        main_logger.info("\n--- Этап 5: Итоги ---")
        if 'df_overall_results' in locals() and df_overall_results is not None and not df_overall_results.empty:
            main_logger.info("  Отображение итогового DataFrame:")
            with pd.option_context('display.max_rows', 20, 'display.max_columns', None, 'display.width', 1000):
                display(df_overall_results.head(100))
            if 'multi_labels' in df_overall_results.columns:
                filled_rows_total = df_overall_results['multi_labels'].apply(lambda x: isinstance(x, list) and len(x) > 0).sum()
                main_logger.info(f"\n  Итого: строк с 'multi_labels': {filled_rows_total:,} из {len(df_overall_results):,}")
                empty_label_rows_df = df_overall_results[df_overall_results['multi_labels'].apply(lambda x: isinstance(x, list) and len(x) == 0)]
                if not empty_label_rows_df.empty:
                    main_logger.info(f"\n  Строки, где 'multi_labels' пусты ({len(empty_label_rows_df):,} шт.):")
                    display(empty_label_rows_df.head())
                else:
                    main_logger.info("  Все строки имеют 'multi_labels'.")
            else:
                main_logger.warning("  Колонка 'multi_labels' отсутствует в итоговом DataFrame.")
        else:
            main_logger.info("  Итоговый DataFrame пуст или не определен.")

    # Остановка слушателя логов
    if log_listener is not None:
        log_listener.stop()
        main_logger.info("Слушатель логов остановлен.")

2025-05-24 15:05:08 - INFO - root - --- Этап 1: Загрузка и фильтрация данных ---
2025-05-24 15:05:09 - INFO - root -   Загружен исходный DataFrame, строк: 186,879
2025-05-24 15:05:09 - INFO - root -   Строк после удаления NaT/пустых текстов: 168,735 (удалено: 18,144)
2025-05-24 15:05:09 - INFO - root -   Строк после фильтрации по годам ([2023, 2024]): 57,353
2025-05-24 15:05:09 - INFO - root -   Подготовлен DataFrame для обработки, строк: 57,353
2025-05-24 15:05:09 - INFO - root - --- Завершена загрузка и фильтрация данных ---

2025-05-24 15:05:09 - INFO - root - --- Этап 2: Загрузка ранее обработанных результатов ---
2025-05-24 15:05:09 - INFO - root -   Файл /Users/maksimlyara/Documents/GitHub/DL_NLP_task/data/raw/processed_news_partial.pkl не найден. Начинаем с нуля.
2025-05-24 15:05:09 - INFO - root - --- Завершена загрузка ранее обработанных результатов ---

2025-05-24 15:05:09 - INFO - root - --- Этап 3: Подготовка к пакетной обработке ---
2025-05-24 15:05:09 - INFO - root -   Вс

/Users/maksimlyara/miniconda3/envs/ikanam_news_env/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Обработка чанков:   0%|          | 0/29 [00:00<?, ?чанк/s]

2025-05-24 15:05:10 - INFO - Воркер-01 - Начинает обработку чанка из 1978 строк (ID 0..1977). Ключ: ..1uew
2025-05-24 15:05:10 - INFO - Воркер-02 - Начинает обработку чанка из 1978 строк (ID 1978..3955). Ключ: ..pVWw
2025-05-24 15:05:10 - INFO - Воркер-03 - Начинает обработку чанка из 1978 строк (ID 3956..5933). Ключ: ..fSgk
2025-05-24 15:05:10 - INFO - Воркер-04 - Начинает обработку чанка из 1978 строк (ID 5934..7911). Ключ: ..NMZQ
2025-05-24 15:05:10 - INFO - Воркер-05 - Начинает обработку чанка из 1978 строк (ID 7912..9889). Ключ: ..8osk
2025-05-24 15:28:36 - INFO - Воркер-01 - Завершил обработку чанка (ID 0..1977). Отправлено API-батчей: 17. Новостей с метками: 1978/1978.
2025-05-24 15:28:36 - INFO - Воркер-06 - Начинает обработку чанка из 1978 строк (ID 9890..11867). Ключ: ..1uew
2025-05-24 15:28:52 - INFO - Воркер-02 - Завершил обработку чанка (ID 1978..3955). Отправлено API-батчей: 16. Новостей с метками: 1978/1978.
2025-05-24 15:28:52 - INFO - Воркер-07 - Начинает обработку чан

,text,multi_labels,hier_label
id,,,
0,▪️ на машину дается скидка в 20% (для Дальнего...,"[Экономика, Общество, Бизнес]","[Экономика, Льготное кредитование]"
1,▪️ новая мера поддержки объединила ранее дейст...,"[Общество, Экономика]","[Общество, Социальная поддержка]"
2,▪️ возобновляется льготное автокредитование (д...,"[Экономика, Общество, Политика]","[Экономика, Социально-экономическая политика]"
3,Украинские войска в новогоднюю ночь обстреляли...,"[Происшествия, Международные отношения, Регион...","[Происшествия, Военные действия]"
4,"Летчики группы ""Вагнер"" рассказали, что даже в...","[Происшествия, Международные отношения]","[Происшествия, Военные действия]"
...,...,...,...
95,Кабмин РФ утвердил льготную ипотечную программ...,"[Экономика, Политика, Региональные новости]","[Экономика, Льготная ипотека]"
96,Минпромторг ожидает роста продаж новых автомоб...,"[Экономика, Бизнес]","[Экономика, Автомобильный рынок]"
97,Идея сокращения новогодних выходных требует со...,"[Общество, Политика]","[Общество, Трудовое законодательство]"


2025-05-24 18:09:30 - INFO - root - 
  Итого: строк с 'multi_labels': 55,844 из 57,353
2025-05-24 18:09:30 - INFO - root - 
  Строки, где 'multi_labels' пусты (1,509 шт.):


,text,multi_labels,hier_label
id,,,
18904,Свежая сводка Минобороны РФ по СВО:\n\n▪️ Даль...,[],[]
18905,"Асад и Путин по телефону обсудили ""пути прекра...",[],[]
18906,Житель Подмосковья получил шесть суток ареста ...,[],[]
18907,"Медведев о словах Байдена про ""подавление"" рук...",[],[]
18908,Путевки из России за границу подорожали за год...,[],[]


2025-05-24 18:09:30 - INFO - root - Слушатель логов остановлен.


## Удаление строк с пустым "hier_label"

In [ ]:
df_filtered = df_overall_results[df_overall_results['hier_label'].apply(lambda x: isinstance(x, list) and len(x) > 0)].copy()
display(df_filtered)
df_filtered.to_csv(DATA_RAW_PATH, index=False)


## Анализ основных классов

In [6]:
df_filtered = pd.read_csv(DATA_RAW_PATH)
df_filtered

,text,multi_labels,hier_label
0,▪️ на машину дается скидка в 20% (для Дальнего...,"['Экономика', 'Общество', 'Бизнес']","['Экономика', 'Льготное кредитование']"
1,▪️ новая мера поддержки объединила ранее дейст...,"['Общество', 'Экономика']","['Общество', 'Социальная поддержка']"
2,▪️ возобновляется льготное автокредитование (д...,"['Экономика', 'Общество', 'Политика']","['Экономика', 'Социально-экономическая политика']"
3,Украинские войска в новогоднюю ночь обстреляли...,"['Происшествия', 'Международные отношения', 'Р...","['Происшествия', 'Военные действия']"
4,"Летчики группы ""Вагнер"" рассказали, что даже в...","['Происшествия', 'Международные отношения']","['Происшествия', 'Военные действия']"
...,...,...,...
55839,Заявка на транзит российского газа через Украи...,"['Экономика', 'Международные отношения', 'Бизн...","['Экономика', 'Энергетика']"
55840,\n\n▪️Российские войска нанесли групповой удар...,"['Происшествия', 'Международные отношения', 'П...","['Происшествия', 'Военные действия']"
55841,"❗️Путин заслушал доклады начальника Генштаба, ...","['Политика', 'Происшествия', 'Общество']","['Политика', 'Внутренняя политика']"
55842,Повторные выбросы нефтепродуктов зафиксированы...,"['Экология', 'Происшествия', 'Региональные нов...","['Экология', 'Загрязнение']"


In [7]:
def safe_literal_eval(s):
    try:
        # Check for NaN and non-string types to avoid errors
        if pd.isna(s) or not isinstance(s, str):
            return [] # Return empty list for missing/non-string values
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        # Handle cases where the string is not a valid list literal
        return []

# Apply ast.literal_eval to convert string representations of lists to actual lists
df_filtered['multi_labels'] = df_filtered['multi_labels'].apply(safe_literal_eval)
df_filtered['hier_label'] = df_filtered['hier_label'].apply(safe_literal_eval)

print("\nDataFrame ПОСЛЕ преобразования типов:")
display(df_filtered.head(10))


# 'multi_labels' - это списки, их нужно преобразовать для Counter, например, в кортежи
# Теперь multi_labels уже содержат actual списки, а не их строковые представления
multi_label_counts = Counter(tuple(sorted(labels)) for labels in df_filtered['multi_labels'] if labels) # sorted для учета порядка
print("\nТоп-10 комбинаций multi_labels:")
for labels, count in multi_label_counts.most_common(10):
    print(f"{labels}: {count}")

# Частота отдельных основных тем (если в multi_labels может быть несколько)
all_single_labels = []
for labels_list in df_filtered['multi_labels']: # Renamed 'labels' to 'labels_list' for clarity
    if labels_list: # Проверка, что список не пуст
        all_single_labels.extend(labels_list)
single_label_counts = Counter(all_single_labels)
print("\nЧастота отдельных основных тем:")
for label, count in single_label_counts.most_common():
    print(f"{label}: {count}")


DataFrame ПОСЛЕ преобразования типов:


,text,multi_labels,hier_label
0,▪️ на машину дается скидка в 20% (для Дальнего...,"[Экономика, Общество, Бизнес]","[Экономика, Льготное кредитование]"
1,▪️ новая мера поддержки объединила ранее дейст...,"[Общество, Экономика]","[Общество, Социальная поддержка]"
2,▪️ возобновляется льготное автокредитование (д...,"[Экономика, Общество, Политика]","[Экономика, Социально-экономическая политика]"
3,Украинские войска в новогоднюю ночь обстреляли...,"[Происшествия, Международные отношения, Регион...","[Происшествия, Военные действия]"
4,"Летчики группы ""Вагнер"" рассказали, что даже в...","[Происшествия, Международные отношения]","[Происшествия, Военные действия]"
5,"Ким Чен Ын заявил, что снаряды из новых пусков...","[Международные отношения, Политика, Технологии]","[Международные отношения, Ядерная программа]"
6,"❗️В центре Донецка , предварительно, из-за обс...","[Происшествия, Региональные новости]","[Происшествия, Обстрелы]"
7,Шесть человек погибли при украинском обстреле ...,"[Происшествия, Региональные новости, Междунаро...","[Происшествия, Военные преступления]"
8,"Северная Корея подтвердила, что и провела ис...","[Международные отношения, Технологии, Политика]","[Международные отношения, Военные испытания]"
9,"Северокорейское агентство ЦТАК пишет, что Пути...","[Международные отношения, Политика]","[Международные отношения, Дипломатические конт..."



Топ-10 комбинаций multi_labels:
('Международные отношения', 'Политика'): 8234
('Происшествия', 'Региональные новости'): 3825
('Международные отношения', 'Политика', 'Происшествия'): 3531
('Международные отношения', 'Происшествия'): 2602
('Международные отношения', 'Политика', 'Экономика'): 2111
('Общество', 'Происшествия', 'Региональные новости'): 2013
('Международные отношения', 'Происшествия', 'Региональные новости'): 1971
('Международные отношения', 'Общество', 'Политика'): 1867
('Политика', 'Происшествия', 'Региональные новости'): 1556
('Общество', 'Политика'): 1448

Частота отдельных основных тем:
Политика: 30431
Международные отношения: 28790
Происшествия: 25829
Региональные новости: 16947
Общество: 16882
Экономика: 7906
Технологии: 5309
Культура: 2843
Бизнес: 2543
Здоровье: 1829
Экология: 1792
Наука: 1520
Спорт: 1188
Образование: 846
Военные действия: 584
Законодательство: 122
Транспорт: 112
История: 58
Оборона: 48
Финансы: 36
Военные новости: 24
Энергетика: 18
Религия: 5
Regio

## Анализ иерархических классов

In [8]:
# 'hier_label' - это списки из двух элементов [тема, подтема]
hier_label_counts = Counter(tuple(labels) for labels in df_filtered['hier_label'] if labels and len(labels) == 2)
print(f"\nТоп-200 иерархических меток (тема, подтема):")
for labels, count in hier_label_counts.most_common(200):
    print(f"{labels}: {count}")


Топ-200 иерархических меток (тема, подтема):
('Происшествия', 'Военные действия'): 4048
('Международные отношения', 'Дипломатия'): 1921
('Политика', 'Внутренняя политика'): 1314
('Политика', 'Законодательство'): 1278
('Политика', 'Выборы'): 1270
('Политика', 'Внешняя политика'): 1258
('Международные отношения', 'Военные действия'): 1251
('Политика', 'Военные действия'): 1176
('Международные отношения', 'Военная помощь'): 992
('Происшествия', 'Обстрелы'): 859
('Происшествия', 'Атаки БПЛА'): 655
('Происшествия', 'Пожары'): 605
('Международные отношения', 'Военный конфликт'): 590
('Происшествия', 'Стихийные бедствия'): 475
('Международные отношения', 'Двусторонние отношения'): 464
('Международные отношения', 'Военные конфликты'): 461
('Происшествия', 'Терроризм'): 451
('Международные отношения', 'Военное сотрудничество'): 437
('Политика', 'Военная политика'): 414
('Экономика', 'Финансы'): 389
('Происшествия', 'Преступность'): 356
('Международные отношения', 'Санкции'): 335
('Технологии',

## Маппинг из словаря и предобработка как для основных классов, так и для иерархических

In [9]:
# 0. Определяем "правильные" основные темы из вашего промпта
VALID_MAIN_LABELS = set([
    "Политика",
    "Экономика",
    "Общество",
    "Происшествия",
    "Спорт",
    "Культура",
    "Технологии",
    "Международные отношения",
    "Региональные новости",
    "Наука",
    "Экология",
    "Здоровье",
    "Образование",
    "Бизнес"
])

# 1. Словарь для переименования "неправильных" тем и их обработки
# Ключ - "неправильная" тема, значение - "правильная" или None для удаления
RENAME_MAP = {
    "Regиональные новости": "Региональные новости",
    "Regionales": "Региональные новости",
    # "Военные действия": None,
    # "Транспорт": None,
    # "Tранспорт": None,
    "Правосудие": "Общество",
    "Медиа": "Общество", 
    "Финансы": "Экономика",
    "История": "Культура",
    "Природа": "Общество",
    "Здравоохранение": "Общество",
    "Криминал": "Происшествия",
    "Безопасность": "Общество",
    "Религия": "Общество",
    "Недвижимость": "Экономика",
    "Футбол": "Спорт",
    "Техника": "Технологии",
    "Мошенничество": "Происшествия",
    "Терроризм": "Происшествия",
    "Ближний Восток": "Международные отношения", 
    "Внутренняя политика": "Политика"
}

def clean_and_normalize_labels_inplace(df: pd.DataFrame) -> None:
    """
    Очищает и нормализует метки в колонках 'multi_labels' и 'hier_label' DataFrame.
    Изменения производятся непосредственно в DataFrame (inplace).
    """
    if 'multi_labels' not in df.columns or 'hier_label' not in df.columns:
        print("Ошибка: DataFrame должен содержать колонки 'multi_labels' и 'hier_label'.")
        return

    processed_rows = 0
    total_rows = len(df)

    for index, row in df.iterrows():
        # Ensure labels are stripped of leading/trailing whitespace for accurate matching
        multi_labels_original = [str(item).strip() for item in row['multi_labels'] if item is not None]
        hier_label_original = [str(item).strip() if item is not None else None for item in row['hier_label']]

        cleaned_multi = []
        
        # 1. Очистка multi_labels
        for label in multi_labels_original:
            if not label: # Skip empty strings resulting from stripping
                continue

            if label in VALID_MAIN_LABELS:
                if label not in cleaned_multi: # Добавляем только уникальные
                    cleaned_multi.append(label)
            elif label in RENAME_MAP:
                new_label = RENAME_MAP[label]
                if new_label and new_label not in cleaned_multi: # Если есть во что переименовать и еще не добавлено
                    cleaned_multi.append(new_label)
            # Если метки нет ни в VALID_MAIN_LABELS, ни в RENAME_MAP, она игнорируется (удаляется)
        
        df.at[index, 'multi_labels'] = cleaned_multi[:3] # Ограничиваем до 3-х и сохраняем

        # 2. Обновление hier_label
        current_first_main_after_cleaning = df.at[index, 'multi_labels'][0] if df.at[index, 'multi_labels'] else None
        
        new_hier_main_label = None
        original_hier_main_label = hier_label_original[0] if hier_label_original else None
        original_hier_sub_label = hier_label_original[1] if hier_label_original and len(hier_label_original) > 1 else None

        if original_hier_main_label: # If there was an original main label
            if original_hier_main_label in VALID_MAIN_LABELS:
                new_hier_main_label = original_hier_main_label
            elif original_hier_main_label in RENAME_MAP:
                new_hier_main_label = RENAME_MAP[original_hier_main_label]
            # If original_hier_main_label is not valid and not remapped, it implicitly becomes None here.

        # If hier_label[0] became None or was None, but multi_labels[0] is valid, use it
        if new_hier_main_label is None and current_first_main_after_cleaning:
            new_hier_main_label = current_first_main_after_cleaning
        
        # Preserve the sub-label only if a main label is established
        new_hier_sub_label = original_hier_sub_label if new_hier_main_label else None

        df.at[index, 'hier_label'] = [new_hier_main_label, new_hier_sub_label]
        
        processed_rows += 1
        if processed_rows % 5000 == 0:
            print(f"Очищено {processed_rows}/{total_rows} строк...")
    
    print(f"Очистка и нормализация меток завершена для {total_rows} строк.")


print("DataFrame ДО очистки:")
display(df_filtered)

# 2. Применяем функцию очистки
clean_and_normalize_labels_inplace(df_filtered)

print("\nDataFrame ПОСЛЕ очистки:")
display(df_filtered)

# 3. Показываем строки, где первая тема multi_labels не соответствует первой теме hier_label
#    (исключая случаи, когда одна из них или обе пусты/None)
mismatched_rows_indices = []
for index, row in df_filtered.iterrows():
    ml = row['multi_labels']
    hl = row['hier_label']

    first_ml = ml[0] if ml else None
    first_hl = hl[0] if hl and hl[0] is not None else None # Убедимся, что hl[0] не None

    if first_ml is not None and first_hl is not None and first_ml != first_hl:
        mismatched_rows_indices.append(index)
    elif (first_ml is not None and first_hl is None and ml): # есть multi, но hier[0] пуст
        mismatched_rows_indices.append(index)

if mismatched_rows_indices:
    print("\nСтроки, где первая тема multi_labels не соответствует первой теме hier_label (или hier_label пуст):")
    display(df_filtered.loc[mismatched_rows_indices])
    df_filtered = df_filtered.drop(mismatched_rows_indices).copy()
else:
    print("\nНет строк, где первая тема multi_labels не соответствует первой теме hier_label.")

# 4. Для проверки, какие темы остались в multi_labels
print("\nЧастота отдельных основных тем ПОСЛЕ ОЧИСТКИ:")
all_single_labels_cleaned = []
for labels_list in df_filtered['multi_labels']:
    if labels_list: # Проверка, что список не пуст
        all_single_labels_cleaned.extend(labels_list)
single_label_counts_cleaned = Counter(all_single_labels_cleaned)
for label, count in single_label_counts_cleaned.most_common():
    print(f"{label}: {count}")

DataFrame ДО очистки:


,text,multi_labels,hier_label
0,▪️ на машину дается скидка в 20% (для Дальнего...,"[Экономика, Общество, Бизнес]","[Экономика, Льготное кредитование]"
1,▪️ новая мера поддержки объединила ранее дейст...,"[Общество, Экономика]","[Общество, Социальная поддержка]"
2,▪️ возобновляется льготное автокредитование (д...,"[Экономика, Общество, Политика]","[Экономика, Социально-экономическая политика]"
3,Украинские войска в новогоднюю ночь обстреляли...,"[Происшествия, Международные отношения, Регион...","[Происшествия, Военные действия]"
4,"Летчики группы ""Вагнер"" рассказали, что даже в...","[Происшествия, Международные отношения]","[Происшествия, Военные действия]"
...,...,...,...
55839,Заявка на транзит российского газа через Украи...,"[Экономика, Международные отношения, Бизнес]","[Экономика, Энергетика]"
55840,\n\n▪️Российские войска нанесли групповой удар...,"[Происшествия, Международные отношения, Полити...","[Происшествия, Военные действия]"
55841,"❗️Путин заслушал доклады начальника Генштаба, ...","[Политика, Происшествия, Общество]","[Политика, Внутренняя политика]"
55842,Повторные выбросы нефтепродуктов зафиксированы...,"[Экология, Происшествия, Региональные новости]","[Экология, Загрязнение]"


Очищено 5000/55844 строк...
Очищено 10000/55844 строк...
Очищено 15000/55844 строк...
Очищено 20000/55844 строк...
Очищено 25000/55844 строк...
Очищено 30000/55844 строк...
Очищено 35000/55844 строк...
Очищено 40000/55844 строк...
Очищено 45000/55844 строк...
Очищено 50000/55844 строк...
Очищено 55000/55844 строк...
Очистка и нормализация меток завершена для 55844 строк.

DataFrame ПОСЛЕ очистки:


,text,multi_labels,hier_label
0,▪️ на машину дается скидка в 20% (для Дальнего...,"[Экономика, Общество, Бизнес]","[Экономика, Льготное кредитование]"
1,▪️ новая мера поддержки объединила ранее дейст...,"[Общество, Экономика]","[Общество, Социальная поддержка]"
2,▪️ возобновляется льготное автокредитование (д...,"[Экономика, Общество, Политика]","[Экономика, Социально-экономическая политика]"
3,Украинские войска в новогоднюю ночь обстреляли...,"[Происшествия, Международные отношения, Регион...","[Происшествия, Военные действия]"
4,"Летчики группы ""Вагнер"" рассказали, что даже в...","[Происшествия, Международные отношения]","[Происшествия, Военные действия]"
...,...,...,...
55839,Заявка на транзит российского газа через Украи...,"[Экономика, Международные отношения, Бизнес]","[Экономика, Энергетика]"
55840,\n\n▪️Российские войска нанесли групповой удар...,"[Происшествия, Международные отношения, Политика]","[Происшествия, Военные действия]"
55841,"❗️Путин заслушал доклады начальника Генштаба, ...","[Политика, Происшествия, Общество]","[Политика, Внутренняя политика]"
55842,Повторные выбросы нефтепродуктов зафиксированы...,"[Экология, Происшествия, Региональные новости]","[Экология, Загрязнение]"



Строки, где первая тема multi_labels не соответствует первой теме hier_label (или hier_label пуст):


,text,multi_labels,hier_label
14,"Власти самопровозглашенного Косово заявили, чт...","[Политика, Международные отношения, Региональн...","[Международные отношения, Региональная безопас..."
57,"Ким Чен Ын, заявивший вчера о необходимости ув...","[Политика, Международные отношения, Технологии]","[Международные отношения, Ядерная программа]"
778,Портрет Путина есть на странице участников сам...,"[Политика, Международные отношения]","[Международные отношения, Международные саммиты]"
1924,Ровно 20 лет назад на заседании Совбеза ООН го...,"[Политика, Международные отношения, Происшествия]","[Международные отношения, Военные конфликты]"
2177,В Париже проходит демонстрация против пенсионн...,"[Политика, Общество, Международные отношения]","[Общество, Протесты]"
...,...,...,...
55759,"США полностью использовали средства, выделенны...","[Политика, Международные отношения, Экономика]","[Международные отношения, Финансовая помощь]"
55762,⚡️Россия вернула из украинского плена 150 воен...,"[Политика, Международные отношения, Общество]","[Международные отношения, Обмен пленными]"
55777,"Ким Чен Ын поздравил ""самого близкого друга и ...","[Политика, Международные отношения]","[Международные отношения, Дипломатия]"
55781,Си Цзиньпин направил Путину поздравление с Нов...,"[Политика, Международные отношения]","[Международные отношения, Дипломатия]"



Частота отдельных основных тем ПОСЛЕ ОЧИСТКИ:
Политика: 29570
Международные отношения: 27719
Происшествия: 25427
Общество: 16282
Региональные новости: 16009
Экономика: 7567
Технологии: 4867
Культура: 2801
Бизнес: 2342
Здоровье: 1709
Экология: 1704
Наука: 1442
Спорт: 1157
Образование: 763


In [10]:
df_filtered

,text,multi_labels,hier_label
0,▪️ на машину дается скидка в 20% (для Дальнего...,"[Экономика, Общество, Бизнес]","[Экономика, Льготное кредитование]"
1,▪️ новая мера поддержки объединила ранее дейст...,"[Общество, Экономика]","[Общество, Социальная поддержка]"
2,▪️ возобновляется льготное автокредитование (д...,"[Экономика, Общество, Политика]","[Экономика, Социально-экономическая политика]"
3,Украинские войска в новогоднюю ночь обстреляли...,"[Происшествия, Международные отношения, Регион...","[Происшествия, Военные действия]"
4,"Летчики группы ""Вагнер"" рассказали, что даже в...","[Происшествия, Международные отношения]","[Происшествия, Военные действия]"
...,...,...,...
55839,Заявка на транзит российского газа через Украи...,"[Экономика, Международные отношения, Бизнес]","[Экономика, Энергетика]"
55840,\n\n▪️Российские войска нанесли групповой удар...,"[Происшествия, Международные отношения, Политика]","[Происшествия, Военные действия]"
55841,"❗️Путин заслушал доклады начальника Генштаба, ...","[Политика, Происшествия, Общество]","[Политика, Внутренняя политика]"
55842,Повторные выбросы нефтепродуктов зафиксированы...,"[Экология, Происшествия, Региональные новости]","[Экология, Загрязнение]"


## Отнесение ИЕРАРХИЧЕСКИХ редких классов в "другое"

In [11]:
# Сначала посчитаем частоту всех иерархических меток
hier_label_tuples = [tuple(labels) for labels in df_filtered['hier_label'] if labels and len(labels) == 2 and all(labels)]
hier_label_counts = Counter(hier_label_tuples)

def consolidate_rare_hier_labels(row):
    hier_label = row['hier_label']
    if hier_label and len(hier_label) == 2 and all(hier_label): # Проверяем, что метка валидна
        current_tuple = tuple(hier_label)
        if hier_label_counts[current_tuple] < MIN_HIER_LABEL_COUNT:
            # Заменяем подтему на "Другое", оставляя основную тему
            return [hier_label[0], "Другое"] 
    return hier_label # Возвращаем как есть, если метка частая или невалидная/пустая

# Применяем только к тем строкам, где hier_label не пустой и корректный
# Создадим новую колонку для безопасности или обновим существующую
df_filtered['hier_label'] = df_filtered.apply(consolidate_rare_hier_labels, axis=1)

# Посмотрим на новые частоты
hier_label_consolidated_tuples = [tuple(labels) for labels in df_filtered['hier_label'] if labels and len(labels) == 2 and all(labels)]
hier_label_consolidated_counts = Counter(hier_label_consolidated_tuples)
print(f"\nТоп иерархических меток ПОСЛЕ консолидации редких менее {MIN_HIER_LABEL_COUNT}:")
for labels, count in hier_label_consolidated_counts.most_common(1000):
    print(f"{labels}: {count}")

# Посмотрим, сколько теперь меток "Другое" для каждой основной темы
other_counts = Counter(item[0] for item in hier_label_consolidated_counts if item[1] == "Другое")
print("\nКоличество подтем 'Другое' для каждой основной темы:")
for main_theme, count in other_counts.items():
     # Ищем общее количество для этой основной темы
    total_for_main_theme = sum(v for k, v in hier_label_consolidated_counts.items() if k[0] == main_theme)
    print(f"{main_theme}: {hier_label_consolidated_counts[(main_theme, 'Другое')]} (из {total_for_main_theme} всего для {main_theme})")


Топ иерархических меток ПОСЛЕ консолидации редких менее 50:
('Происшествия', 'Другое'): 4545
('Происшествия', 'Военные действия'): 4047
('Политика', 'Другое'): 3328
('Международные отношения', 'Другое'): 3257
('Общество', 'Другое'): 2447
('Международные отношения', 'Дипломатия'): 1826
('Экономика', 'Другое'): 1675
('Политика', 'Внутренняя политика'): 1313
('Политика', 'Законодательство'): 1277
('Политика', 'Выборы'): 1269
('Политика', 'Внешняя политика'): 1258
('Международные отношения', 'Военные действия'): 1249
('Политика', 'Военные действия'): 1176
('Международные отношения', 'Военная помощь'): 982
('Культура', 'Другое'): 875
('Происшествия', 'Обстрелы'): 865
('Технологии', 'Другое'): 845
('Региональные новости', 'Другое'): 657
('Происшествия', 'Атаки БПЛА'): 656
('Происшествия', 'Пожары'): 605
('Международные отношения', 'Военный конфликт'): 583
('Происшествия', 'Стихийные бедствия'): 475
('Происшествия', 'Терроризм'): 449
('Международные отношения', 'Военные конфликты'): 449
('Ме

In [12]:
df_filtered.to_csv(DATA_PREPROCESSED_PATH, index=False)

## Блок с объединением похожий названий ИЕРАРХИЧЕСКИХ классов с помощью эмбеддингов и их косинусного расстояния

In [13]:
df_filtered = pd.read_csv(DATA_PREPROCESSED_PATH)

# 0. Параметры
print(f"Загрузка модели для эмбеддингов: {MODEL_NAME_EMBEDDINGS}...")
try:
    embedder = SentenceTransformer(MODEL_NAME_EMBEDDINGS)
    print("Модель загружена.")
except Exception as e:
    print(f"Ошибка загрузки модели эмбеддингов: {e}")
    print("Дальнейшее объединение подтем по семантической близости невозможно без модели.")
    exit()


def merge_similar_subthemes_in_dataframe(df, hier_label_col='hier_label'):
    """
    Объединяет семантически похожие подтемы внутри каждой основной темы в DataFrame.
    Изменения производятся в новой колонке 'hier_label_merged'.
    """
    if hier_label_col not in df.columns:
        print(f"Ошибка: Колонка '{hier_label_col}' отсутствует в DataFrame.")
        return df

    df_copy = df.copy()
    # Создаем новую колонку, чтобы не изменять исходную во время итераций по ней же
    df_copy['hier_label_merged'] = df_copy[hier_label_col].copy().astype(object)

    # 1. Собираем все уникальные пары (основная_тема, подтема) и их частоты
    valid_hier_labels = [tuple(labels) for labels in df_copy[hier_label_col]
                         if isinstance(labels, list) and len(labels) == 2 and all(isinstance(l, str) and l is not None for l in labels)]
    hier_label_counts = Counter(valid_hier_labels)

    # 2. Группируем подтемы по основным темам
    main_theme_to_subthemes = {}
    for (main_theme, sub_theme), count in hier_label_counts.items():
        if main_theme not in main_theme_to_subthemes:
            main_theme_to_subthemes[main_theme] = []
        main_theme_to_subthemes[main_theme].append({'subtheme': sub_theme, 'count': count, 'original_tuple': (main_theme, sub_theme)})

    print(f"\nНачинаем объединение похожих подтем. Порог сходства: {SIMILARITY_THRESHOLD}")

    # 3. Для каждой основной темы объединяем похожие подтемы
    final_rename_map_for_subthemes = {} # (main, old_sub) -> (main, new_sub)

    for main_theme, subtheme_dicts in main_theme_to_subthemes.items():
        # print(f"DEBUG: Processing main_theme: {main_theme}, subtheme_dicts count: {len(subtheme_dicts)}")
        if len(subtheme_dicts) < 2: # Нечего объединять, если подтема одна или нет
            continue

        # Исключаем "Другое" из процесса объединения (и подтемы с низким счетчиком, если они не являются целью)
        subthemes_to_process = [st for st in subtheme_dicts if st['subtheme'] != "Другое"]
        
        if len(subthemes_to_process) < 2:
            continue

        print(f"\nОбработка основной темы: {main_theme} (уникальных подтем, кроме 'Другое': {len(subthemes_to_process)})")

        # Сортируем подтемы по убыванию частоты, чтобы более частые были приоритетными целями для слияния
        subthemes_to_process.sort(key=lambda x: x['count'], reverse=True)
        
        subtheme_names = [st['subtheme'] for st in subthemes_to_process]
        
        if not subtheme_names: continue

        embeddings = embedder.encode(subtheme_names, show_progress_bar=False)
        cosine_matrix = cosine_similarity(embeddings)
        
        merged_source_subthemes_names = set() # Чтобы отслеживать подтемы, которые УЖЕ БЫЛИ СЛИТЫ в другие
        
        # Итеративно объединяем. Мы хотим объединять более редкие в более частые.
        # Проходим по всем парам (i, j) где i - потенциальная цель (более частая), j - потенциальный источник (менее частая)
        for i in range(len(subthemes_to_process)):
            sub_i_info = subthemes_to_process[i]
            # Если текущая (более частая) подтема сама уже была слита в другую (по логике, это не должно произойти,
            # но как дополнительная защита)
            if sub_i_info['subtheme'] in merged_source_subthemes_names:
                continue 

            for j in range(i + 1, len(subthemes_to_process)): # j всегда будет менее частой, чем i, или равной
                sub_j_info = subthemes_to_process[j]

                # Если подтема-источник уже была слита, пропускаем
                if sub_j_info['subtheme'] in merged_source_subthemes_names:
                    continue

                similarity = cosine_matrix[i, j]

                if similarity >= SIMILARITY_THRESHOLD:
                    target_info = sub_i_info  # i - более частая (или равная), поэтому она цель
                    source_info = sub_j_info  # j - менее частая, поэтому она источник

                    # Дополнительное условие: не объединяем, если целевая подтема сама по себе слишком редкая
                    # и при этом не является значительно более частой, чем источник.
                    # Это предотвращает создание новых "мусорных" кластеров из редких подтем.
                    if target_info['count'] < MIN_SUBTHEME_COUNT_FOR_MERGE_TARGET and source_info['count'] > target_info['count'] / 2:
                        continue # Если цель слишком редка и источник не намного реже, лучше не объединять.

                    print(f"    Объединяем: '{source_info['subtheme']}' ({source_info['count']}) -> '{target_info['subtheme']}' ({target_info['count']}) (сходство: {similarity:.3f})")
                    final_rename_map_for_subthemes[source_info['original_tuple']] = target_info['original_tuple']
                    merged_source_subthemes_names.add(source_info['subtheme']) # Добавляем имя подтемы, которую объединили

    # 4. Применяем созданный final_rename_map_for_subthemes к DataFrame
    if final_rename_map_for_subthemes:
        print(f"\nПрименение {len(final_rename_map_for_subthemes)} правил объединения подтем к DataFrame...")
        
        def apply_subtheme_merging(hier_label_list):
            # Ensure it's a list with 2 elements and both are strings (not None)
            if not (isinstance(hier_label_list, list) and len(hier_label_list) == 2 and \
                    all(isinstance(l, str) and l is not None for l in hier_label_list)):
                return [None, None] # Return default for invalid formats

            current_tuple = tuple(hier_label_list)
            # Идем по цепочке переименований, если есть (A->B, B->C => A->C)
            # Необходимо проверять наличие в словаре, чтобы избежать KeyError
            temp_tuple = current_tuple
            while temp_tuple in final_rename_map_for_subthemes:
                temp_tuple = final_rename_map_for_subthemes[temp_tuple]
                # Дополнительная проверка на зацикливание (хотя в данной логике маловероятно)
                if temp_tuple == current_tuple:
                    break # Break if a loop is detected
            return list(temp_tuple)
            
        df_copy['hier_label_merged'] = df_copy['hier_label_merged'].apply(apply_subtheme_merging)
        print("Применение объединения подтем завершено.")
    else:
        print("Не найдено подтем для семантического объединения с текущим порогом.")
        
    return df_copy


if 'hier_label' not in df_filtered.columns:
    print("Колонка 'hier_label' не найдена. Пожалуйста, создайте ее сначала (объединив редкие подтемы в 'Другое').")
else:
    print("DataFrame ДО семантического объединения подтем (на основе 'hier_label'):")
    hier_counts_before_semantic = Counter(tuple(l) for l in df_filtered['hier_label'] if isinstance(l, list) and len(l)==2 and all(l))
    print("Топ-30 иерархических меток ДО семантического объединения:")
    for labels, count in hier_counts_before_semantic.most_common(30):
            print(f"{labels}: {count}")


    # Применяем семантическое объединение
    df_filtered_after_semantic_merge = merge_similar_subthemes_in_dataframe(df_filtered, hier_label_col='hier_label')

    print("\nDataFrame ПОСЛЕ семантического объединения подтем:")
    display(df_filtered_after_semantic_merge[['hier_label', 'hier_label_merged']].head(20))
    
    hier_counts_after_semantic = Counter(tuple(l) for l in df_filtered_after_semantic_merge['hier_label_merged'] if isinstance(l, list) and len(l)==2 and all(l))
    print("\nТоп-30 иерархических меток ПОСЛЕ семантического объединения:")
    for labels, count in hier_counts_after_semantic.most_common(30):
            print(f"{labels}: {count}")
    
    # Если хотите обновить исходную колонку в df_filtered:
    df_filtered['hier_label'] = df_filtered_after_semantic_merge['hier_label_merged']
    df_filtered.drop(columns=['hier_label_merged'], inplace=True, errors='ignore')

    # Проверка, сколько уникальных осталось
    num_unique_before = len(hier_counts_before_semantic)
    num_unique_after = len(hier_counts_after_semantic)
    print(f"\nКоличество уникальных иерархических меток: {num_unique_before} -> {num_unique_after}")

    # Показать примеры, где произошло объединение
    # Создадим столбец с исходным значением для сравнения, если 'hier_label_merged' отличается
    df_filtered_after_semantic_merge['temp_hier_consolidated_tuple'] = df_filtered_after_semantic_merge['hier_label'].apply(lambda x: tuple(x) if isinstance(x,list) and len(x)==2 else None)
    df_filtered_after_semantic_merge['temp_hier_merged_tuple'] = df_filtered_after_semantic_merge['hier_label_merged'].apply(lambda x: tuple(x) if isinstance(x,list) and len(x)==2 else None)
    
    df_changed_semantic = df_filtered_after_semantic_merge[
        df_filtered_after_semantic_merge['temp_hier_consolidated_tuple'].notna() &
        df_filtered_after_semantic_merge['temp_hier_merged_tuple'].notna() &
        (df_filtered_after_semantic_merge['temp_hier_consolidated_tuple'] != df_filtered_after_semantic_merge['temp_hier_merged_tuple'])
    ]
    if not df_changed_semantic.empty:
        print(f"\nПримеры строк, где подтемы были семантически объединены ({len(df_changed_semantic)} строк):")
        # Выведем только те колонки, которые наглядно показывают изменения
        display(df_changed_semantic[['text', 'hier_label', 'hier_label_merged']])
    else:
        print("\nНе найдено строк, где подтемы были семантически объединены (возможно, порог сходства слишком высок или данные слишком однородны).")
    
    df_filtered_after_semantic_merge = df_filtered_after_semantic_merge.drop(columns=['temp_hier_consolidated_tuple', 'temp_hier_merged_tuple'], errors='ignore')

df_filtered.to_csv(DATA_PREPROCESSED_PATH, index=False)

Загрузка модели для эмбеддингов: sergeyzh/BERTA...


Default prompt name is set to 'Classification'. This prompt will be applied to all `encode()` calls, except if `encode()` is called with `prompt` or `prompt_name` parameters.


Модель загружена.
DataFrame ДО семантического объединения подтем (на основе 'hier_label'):
Топ-30 иерархических меток ДО семантического объединения:

Начинаем объединение похожих подтем. Порог сходства: 0.8
Не найдено подтем для семантического объединения с текущим порогом.

DataFrame ПОСЛЕ семантического объединения подтем:


,hier_label,hier_label_merged
0,"['Экономика', 'Другое']","['Экономика', 'Другое']"
1,"['Общество', 'Социальная поддержка']","['Общество', 'Социальная поддержка']"
2,"['Экономика', 'Другое']","['Экономика', 'Другое']"
3,"['Происшествия', 'Военные действия']","['Происшествия', 'Военные действия']"
4,"['Происшествия', 'Военные действия']","['Происшествия', 'Военные действия']"
5,"['Международные отношения', 'Другое']","['Международные отношения', 'Другое']"
6,"['Происшествия', 'Обстрелы']","['Происшествия', 'Обстрелы']"
7,"['Происшествия', 'Военные преступления']","['Происшествия', 'Военные преступления']"
8,"['Международные отношения', 'Другое']","['Международные отношения', 'Другое']"
9,"['Международные отношения', 'Другое']","['Международные отношения', 'Другое']"



Топ-30 иерархических меток ПОСЛЕ семантического объединения:

Количество уникальных иерархических меток: 0 -> 0

Не найдено строк, где подтемы были семантически объединены (возможно, порог сходства слишком высок или данные слишком однородны).


In [14]:
df_filtered

,text,multi_labels,hier_label
0,▪️ на машину дается скидка в 20% (для Дальнего...,"['Экономика', 'Общество', 'Бизнес']","['Экономика', 'Другое']"
1,▪️ новая мера поддержки объединила ранее дейст...,"['Общество', 'Экономика']","['Общество', 'Социальная поддержка']"
2,▪️ возобновляется льготное автокредитование (д...,"['Экономика', 'Общество', 'Политика']","['Экономика', 'Другое']"
3,Украинские войска в новогоднюю ночь обстреляли...,"['Происшествия', 'Международные отношения', 'Р...","['Происшествия', 'Военные действия']"
4,"Летчики группы ""Вагнер"" рассказали, что даже в...","['Происшествия', 'Международные отношения']","['Происшествия', 'Военные действия']"
...,...,...,...
55193,Заявка на транзит российского газа через Украи...,"['Экономика', 'Международные отношения', 'Бизн...","['Экономика', 'Энергетика']"
55194,\n\n▪️Российские войска нанесли групповой удар...,"['Происшествия', 'Международные отношения', 'П...","['Происшествия', 'Военные действия']"
55195,"❗️Путин заслушал доклады начальника Генштаба, ...","['Политика', 'Происшествия', 'Общество']","['Политика', 'Внутренняя политика']"
55196,Повторные выбросы нефтепродуктов зафиксированы...,"['Экология', 'Происшествия', 'Региональные нов...","['Экология', 'Другое']"
